In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from numba import njit
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_colwidth',300)
import random
import math

In [19]:
@njit
def simulate_stock_price(s, dt, v, ticks):
    results = [s]
    for j in range(ticks):
        results.append(round(results[-1] * math.exp(-v**2/2 * dt + v * math.sqrt(dt) * random.normalvariate(0, 1)),2))
#         results.append(round(results[-1] * math.exp(-v**2/2 * dt + v * math.sqrt(dt) * np.random.normal(0, 1)),2))
    return results

@njit
def mc_GO(S, higher, lower, ticks, runs, dt, sigma):
    price = 0
    for i in range(runs):
        path = simulate_stock_price(S, dt, sigma, ticks)
        if max(path) >= higher or min(path) <= lower:
            price += 1
    return price/runs

@njit
def mc_SB(S, higher, lower, ticks, runs, dt, sigma):
    return 1 - mc_GO(S, higher, lower, ticks, runs, dt, sigma)

In [20]:
def determine_K(mu_new, sigma, h, x, t, mu):
    delta = (1 / 100000) * np.exp(-((mu - 0.5 * sigma ** 2) * (h - x)) / sigma ** 2) * (h ** 2) / (np.pi * sigma ** 2)
    A = (mu_new ** 2) / (2 * sigma ** 2)
    B = (np.pi ** 2 * sigma ** 2) / (2 * h ** 2)
    numerator = max(1, np.exp(-A * t) / (B * delta))
    denominator = B * t
    K = int(np.sqrt(np.log(numerator / denominator)))
#     print(K)
#     return K
    return min(max(K, 16), 1000)  # Ensure K is within [16, 1000]

def stability_constant(mu, sigma, eta, h, x, epsilon=1e-8):
    exponent = 1 - ((mu - 0.5 * sigma ** 2) * (eta * h - x)) / sigma ** 2
    numerator = np.exp(exponent)
    denominator = epsilon * (np.exp(1) * (np.pi + 1) + max(0, (mu - 0.5 * sigma ** 2) * (eta * h - x) * np.pi / sigma ** 2))
    return (1 / 100000) * (numerator / denominator)

def common_function_pelsser(eta, S, U, D, t, mu, sigma, r_q, w, epsilon=1e-8):
    h = np.log(U / D)
    
    # Determine x based on eta
    x = np.log(S / D) if eta == 1 else np.log(U / S)
    
    # Calculate mu_new
    mu_new = np.sqrt(max(0, (mu - 0.5 * sigma ** 2) ** 2 + 2 * r_q * sigma ** 2 * (1 - w)))
#     print(mu_new)
    # Determine K
    K = determine_K(mu_new, sigma, h, x, t, mu)
    # Calculate the first part of the common function (hyperbolic sine term)
    first_term = np.sinh((mu_new / sigma ** 2) * x) / np.sinh((mu_new / sigma ** 2) * h)
    
    # Summation term
    sum_term = 0
    for k in range(1, K + 1):
        lambda_k = 0.5 * ((mu_new ** 2 / sigma ** 2) + (k ** 2 * np.pi ** 2 * sigma ** 2 / h ** 2))
        phi = ((sigma ** 2 * k) / (h ** 2 * lambda_k)) * np.exp(-lambda_k * t)
        sum_term += phi * np.pi * np.sin(k * np.pi * (h - x) / h)
    
    # Check stability condition if K=1
    if K == 1:
        phi = (sigma ** 2 * 1) / (h ** 2 * (0.5 * ((mu_new ** 2 / sigma ** 2) + (1 ** 2 * np.pi ** 2 * sigma ** 2 / h ** 2)))) * np.exp(-0.5 * ((mu_new ** 2 / sigma ** 2) + (1 ** 2 * np.pi ** 2 * sigma ** 2 / h ** 2)) * t)
        stab_const = stability_constant(mu, sigma, eta, h, x, epsilon)
        if abs(phi) > stab_const:
            raise ValueError("Stability condition violated: |phi| exceeds the stability constant.")
    
    # Return the full common function value
    return np.exp(-r_q * t * w) * (first_term - sum_term)


def ot_up_ko_down(S, U, D, t, mu, sigma, r_q, w):
    h = np.log(U / D)
    x = np.log(S / D)
    exponent = (mu - 0.5 * sigma ** 2) * (h - x) / sigma ** 2
    return np.exp(exponent) * common_function_pelsser(eta=1, S=S, U=U, D=D, t=t, mu=mu, sigma=sigma, r_q=r_q, w=w)

def ot_down_ko_up(S, U, D, t, mu, sigma, r_q, w):
    x = np.log(U / S)
    exponent = (mu - 0.5 * sigma ** 2) * x / sigma ** 2
    return np.exp(exponent) * common_function_pelsser(eta=0, S=S, U=U, D=D, t=t, mu=mu, sigma=sigma, r_q=r_q, w=w)

def up_or_down(S, U, D, t, sigma, r_q, w, mu):
    ot_up_ko_down_price = ot_up_ko_down(S, U, D, t, mu, sigma, r_q, w)
    ot_down_ko_up_price = ot_down_ko_up(S, U, D, t, mu, sigma, r_q, w)
#     print(ot_up_ko_down_price,ot_down_ko_up_price)
    return ot_up_ko_down_price + ot_down_ko_up_price


In [59]:
def compute_prices(df, S, higher, lower, ticks, stake, runs=100000, sigma=1, mu=0, r_q=0, w=0, comm=1.2):

    t = ticks/365/86400
    beta = 0.5826
    dt = 1 / (365 * 86400)
    U = higher * np.exp(1 * beta * sigma * np.sqrt(dt))
    D = lower * np.exp(-1 * beta * sigma * np.sqrt(dt))
    comm /= 100

    price_GO = up_or_down(S, U, D, t, sigma, r_q, w, mu=0)
    payout_GO = stake/(price_GO+comm)
    
    price_SB = 1 - price_GO
    payout_SB = stake/(price_SB+comm)
    
    price_GO_mc = mc_GO(S, higher, lower, ticks, runs, dt, sigma)
    payout_GO_mc = stake/(price_GO_mc+comm)
    
    price_SB_mc = 1 - price_GO_mc
    payout_SB_mc = stake/(price_SB_mc+comm)

    # Create a DataFrame
#     df = pd.DataFrame()

    row = len(df)-1
    # Assign variables to the DataFrame
    df.at[row, 'price_GO'] = price_GO
    df.at[row, 'price_GO_mc'] = price_GO_mc
    df.at[row, 'diff_GO'] = (price_GO_mc - price_GO) * 100
    df.at[row, 'payout_GO'] = payout_GO
    df.at[row, 'payout_GO_mc'] = payout_GO_mc
    df.at[row, 'price_SB'] = price_SB
    df.at[row, 'price_SB_mc'] = price_SB_mc
    df.at[row, 'diff_SB'] = (price_SB_mc - price_SB) * 100
    df.at[row, 'payout_SB'] = payout_SB
    df.at[row, 'payout_SB_mc'] = payout_SB_mc

    display(df)

In [63]:
df = pd.DataFrame(columns = ['spot', 'high_bar', 'low_bar', 'ticks', 'stake', 'mc_runs', 'price_GO', 'price_GO_mc', 'diff_GO', \
                             'payout_GO', 'payout_GO_mc', 'price_SB', 'price_SB_mc', 'diff_SB', 'payout_SB', 'payout_SB_mc'])

In [ ]:
# Sample parameters
S = 450
bar_diff = 200
higher = S+bar_diff
lower = S-bar_diff
ticks = 2 * 30 * 86400
stake = 100
runs = 3000
row = len(df)
df.at[row, 'spot'] = S
df.at[row, 'high_bar'] = higher
df.at[row, 'low_bar'] = lower
df.at[row, 'ticks'] = ticks
df.at[row, 'stake'] = stake
df.at[row, 'mc_runs'] = runs
compute_prices(df=df, S=S, higher=higher, lower=lower, ticks=ticks, stake=stake, runs=runs)